In [1]:
from datasets import load_dataset

data_files = {
    "train": "/data/wzq/cache/huggingface/hub/datasets--code-search-net--code_search_net/snapshots/fdc6a9e39575768c27eb8a2a5f702bf846eb4759/data/python/final/jsonl/train/*",
    "test": "/data/wzq/cache/huggingface/hub/datasets--code-search-net--code_search_net/snapshots/fdc6a9e39575768c27eb8a2a5f702bf846eb4759/data/python/final/jsonl/test/*",
    "valid": "/data/wzq/cache/huggingface/hub/datasets--code-search-net--code_search_net/snapshots/fdc6a9e39575768c27eb8a2a5f702bf846eb4759/data/python/final/jsonl/valid/*"}

raw_datasets = load_dataset("json", data_files=data_files)

In [2]:
raw_datasets["train"]

Dataset({
    features: ['repo', 'path', 'func_name', 'original_string', 'language', 'code', 'code_tokens', 'docstring', 'docstring_tokens', 'sha', 'url', 'partition'],
    num_rows: 412178
})

In [3]:
raw_datasets["train"][123456]["original_string"] == raw_datasets["train"][123456]["code"]

True

In [4]:
raw_datasets["valid"]

Dataset({
    features: ['repo', 'path', 'func_name', 'original_string', 'language', 'code', 'code_tokens', 'docstring', 'docstring_tokens', 'sha', 'url', 'partition'],
    num_rows: 23107
})

In [5]:
raw_datasets.filter(lambda x: x["original_string"] != x["code"])

Filter:   0%|          | 0/412178 [00:00<?, ? examples/s]

Filter:   0%|          | 0/22176 [00:00<?, ? examples/s]

Filter:   0%|          | 0/23107 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['repo', 'path', 'func_name', 'original_string', 'language', 'code', 'code_tokens', 'docstring', 'docstring_tokens', 'sha', 'url', 'partition'],
        num_rows: 0
    })
    test: Dataset({
        features: ['repo', 'path', 'func_name', 'original_string', 'language', 'code', 'code_tokens', 'docstring', 'docstring_tokens', 'sha', 'url', 'partition'],
        num_rows: 0
    })
    valid: Dataset({
        features: ['repo', 'path', 'func_name', 'original_string', 'language', 'code', 'code_tokens', 'docstring', 'docstring_tokens', 'sha', 'url', 'partition'],
        num_rows: 0
    })
})

In [12]:
raw_datasets.filter(
    lambda x: [a != b for a, b in zip(x["original_string"], x["code"])],
    batched=True
)

Filter:   0%|          | 0/412178 [00:00<?, ? examples/s]

Filter:   0%|          | 0/22176 [00:00<?, ? examples/s]

Filter:   0%|          | 0/23107 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['repo', 'path', 'func_name', 'original_string', 'language', 'code', 'code_tokens', 'docstring', 'docstring_tokens', 'sha', 'url', 'partition'],
        num_rows: 0
    })
    test: Dataset({
        features: ['repo', 'path', 'func_name', 'original_string', 'language', 'code', 'code_tokens', 'docstring', 'docstring_tokens', 'sha', 'url', 'partition'],
        num_rows: 0
    })
    valid: Dataset({
        features: ['repo', 'path', 'func_name', 'original_string', 'language', 'code', 'code_tokens', 'docstring', 'docstring_tokens', 'sha', 'url', 'partition'],
        num_rows: 0
    })
})

In [6]:
print(raw_datasets["train"][123456]["original_string"])

def last_rate_limit(self):
        """
        A `dict` of the rate limit information returned in the most recent
        response, or `None` if no requests have been made yet.  The `dict`
        consists of all headers whose names begin with ``"RateLimit"`` (case
        insensitive).

        The DigitalOcean API specifies the following rate limit headers:

        :var string RateLimit-Limit: the number of requests that can be made
            per hour
        :var string RateLimit-Remaining: the number of requests remaining until
            the limit is reached
        :var string RateLimit-Reset: the Unix timestamp for the time when the
            oldest request will expire from rate limit consideration
        """
        if self.last_response is None:
            return None
        else:
            return {k:v for k,v in iteritems(self.last_response.headers)
                        if k.lower().startswith('ratelimit')}


In [7]:
training_corpus = (raw_datasets["train"][i:i + 1000]["original_string"]
                   for i in range(0, len(raw_datasets["train"]), 1000))

In [8]:
gen = (i for i in range(10))
print(list(gen))
print(list(gen))

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
[]


In [9]:
# def get_training_corpus():
#     return (
#         raw_datasets["train"][i:i + 1000]["original_string"]
#         for i in range(0, len(raw_datasets["train"]), 1000)
#     )

def get_training_corpus():
    dataset = raw_datasets["train"]
    for start_idx in range(0, len(data_files), 1000):
        samples = dataset[start_idx:start_idx + 1000]
        yield samples["original_string"]

In [10]:
from transformers import AutoTokenizer

old_tokenizer = AutoTokenizer.from_pretrained("gpt2")

In [11]:
example = '''def add_numbers(a, b):
    """Add the two numbers `a` and `b`."""
    return a + b'''

tokens = old_tokenizer.tokenize(example)
tokens

['def',
 'Ġadd',
 '_',
 'n',
 'umbers',
 '(',
 'a',
 ',',
 'Ġb',
 '):',
 'Ċ',
 'Ġ',
 'Ġ',
 'Ġ',
 'Ġ"""',
 'Add',
 'Ġthe',
 'Ġtwo',
 'Ġnumbers',
 'Ġ`',
 'a',
 '`',
 'Ġand',
 'Ġ`',
 'b',
 '`',
 '."',
 '""',
 'Ċ',
 'Ġ',
 'Ġ',
 'Ġ',
 'Ġreturn',
 'Ġa',
 'Ġ+',
 'Ġb']

In [13]:
%time
tokenizer = old_tokenizer.train_new_from_iterator(training_corpus, 52000)




CPU times: user 54min 7s, sys: 3min 39s, total: 57min 47s
Wall time: 1min 5s


In [15]:
tokens = tokenizer.tokenize(example)
tokens

['def',
 'Ġadd',
 '_',
 'numbers',
 '(',
 'a',
 ',',
 'Ġb',
 '):',
 'ĊĠĠĠ',
 'Ġ"""',
 'Add',
 'Ġthe',
 'Ġtwo',
 'Ġnumbers',
 'Ġ`',
 'a',
 '`',
 'Ġand',
 'Ġ`',
 'b',
 '`."""',
 'ĊĠĠĠ',
 'Ġreturn',
 'Ġa',
 'Ġ+',
 'Ġb']

In [16]:
print(len(tokens))
print(len(old_tokenizer.tokenize(example)))

27
36


In [17]:
example = """class LinearLayer():
    def __init__(self, input_size, output_size):
        self.weight = torch.randn(input_size, output_size)
        self.bias = torch.zeros(output_size)

    def __call__(self, x):
        return x @ self.weights + self.bias
    """
tokenizer.tokenize(example)

['class',
 'ĠLinear',
 'Layer',
 '():',
 'ĊĠĠĠ',
 'Ġdef',
 'Ġ__',
 'init',
 '__(',
 'self',
 ',',
 'Ġinput',
 '_',
 'size',
 ',',
 'Ġoutput',
 '_',
 'size',
 '):',
 'ĊĠĠĠĠĠĠĠ',
 'Ġself',
 '.',
 'weight',
 'Ġ=',
 'Ġtorch',
 '.',
 'randn',
 '(',
 'input',
 '_',
 'size',
 ',',
 'Ġoutput',
 '_',
 'size',
 ')',
 'ĊĠĠĠĠĠĠĠ',
 'Ġself',
 '.',
 'bias',
 'Ġ=',
 'Ġtorch',
 '.',
 'zeros',
 '(',
 'output',
 '_',
 'size',
 ')',
 'ĊĊĠĠĠ',
 'Ġdef',
 'Ġ__',
 'call',
 '__(',
 'self',
 ',',
 'Ġx',
 '):',
 'ĊĠĠĠĠĠĠĠ',
 'Ġreturn',
 'Ġx',
 'Ġ@',
 'Ġself',
 '.',
 'weights',
 'Ġ+',
 'Ġself',
 '.',
 'bias',
 'ĊĠĠĠĠ']

In [20]:
tokenizer.save_pretrained("/data/wzq/tmp/hugging-face-nlp-course/course/zh-CN/chapter6/code-search-net-tokenizer")

('/data/wzq/tmp/hugging-face-nlp-course/course/zh-CN/chapter6/code-search-net-tokenizer/tokenizer_config.json',
 '/data/wzq/tmp/hugging-face-nlp-course/course/zh-CN/chapter6/code-search-net-tokenizer/special_tokens_map.json',
 '/data/wzq/tmp/hugging-face-nlp-course/course/zh-CN/chapter6/code-search-net-tokenizer/vocab.json',
 '/data/wzq/tmp/hugging-face-nlp-course/course/zh-CN/chapter6/code-search-net-tokenizer/merges.txt',
 '/data/wzq/tmp/hugging-face-nlp-course/course/zh-CN/chapter6/code-search-net-tokenizer/added_tokens.json',
 '/data/wzq/tmp/hugging-face-nlp-course/course/zh-CN/chapter6/code-search-net-tokenizer/tokenizer.json')

In [21]:
from huggingface_hub import notebook_login

notebook_login()

In [22]:
tokenizer.push_to_hub("code-search-net-tokenizer")

CommitInfo(commit_url='https://huggingface.co/KinomotoMio/code-search-net-tokenizer/commit/6f79a1fcf3dec63cd7ebf4c836dc9b37c6582e38', commit_message='Upload tokenizer', commit_description='', oid='6f79a1fcf3dec63cd7ebf4c836dc9b37c6582e38', pr_url=None, repo_url=RepoUrl('https://hf-mirror.com/KinomotoMio/code-search-net-tokenizer', endpoint='https://hf-mirror.com', repo_type='model', repo_id='KinomotoMio/code-search-net-tokenizer'), pr_revision=None, pr_num=None)

In [23]:
# 将下面的 "huggingface-course" 替换为你的用户名来加载你的 tokenizer
tokenizer = AutoTokenizer.from_pretrained("KinomotoMio/code-search-net-tokenizer")

tokenizer_config.json:   0%|          | 0.00/471 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]